In [31]:
import fiona


# Open shapefile for reading election results of 2016
fh = fiona.open('elec2016_1.shp', 'r')
TotalWastedDemocrat=0
TotalWastedRep=0
TotalVotes=0
for f in fh:
    pctname = f['properties']['PCTNAME'] #get the precint name
    repVotes = f['properties']['USPRSR'] #get the republican votes
    demVotes = f['properties']['USPRSDFL'] #get the democrat votes
    totVotes = repVotes + demVotes #total votes by precints of Democrat and Republican Party
    TotalVotes=TotalVotes+totVotes #overall total votes (Republican+Democrat)
    if repVotes > demVotes: #if republican won
        wastedRVotes = int(( (repVotes - demVotes)/2) - 1) #wasted republican votes
        wastedDVotes = demVotes #wasted democrat votes
        favor='efficiency gap is in favor of Republicans'
    elif demVotes > repVotes: #if democrat won
        wastedDVotes = int(( (demVotes - repVotes)/2) - 1) #wasted democrat votes
        wastedRVotes = repVotes #wasted republican votes
        favor='efficiency gap is in favor of Democrats'
    if totVotes!=0: #division by zero error eliminated here for the following formula
        effGapbyPre = abs((wastedRVotes - wastedDVotes)/totVotes)*100 #efficiency gap by precints (percentage)
    TotalWastedDemocrat=TotalWastedDemocrat+wastedDVotes # total wasted votes by democrats
    TotalWastedRep=TotalWastedRep+wastedRVotes #total wasted votes by republicans
fh.close # Close the file
effGap=abs((TotalWastedDemocrat - TotalWastedRep)/TotalVotes)*100 #overall efficiency gap (percentage)
print("efficiency gap", effGap) 

efficiency gap 2.3307093887651864


In [1]:
import fiona
import folium
import json
# Open shapefile for reading (Census Tracts in Minnesota), save as file handle
fh = fiona.open('elec2016_1.shp', 'r')
m = folium.Map(location=[45.00, -90.78], zoom_start=6) 
# Create a folium map# Make a counter, then loop over features and map out the first 100
# WARNING: Folium can crash when trying to map all the tracts. It takes a beefy machine to do it so be warned. :)
count = 0
for f in fh: # Loop over features (f) from shapefile (fh)
    j = json.dumps(f) # Dump the GeoJSON from the feature (look this up)
    folium.features.GeoJson(j).add_to(m) # Add it to the map
    count += 1
    if count > 10:
        break
fh.close()
m